https://www.maricopacountyattorney.org/CivicAlerts.aspx?AID=400

Crime reports?


In [ ]:
var fs = require('fs');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = path.join(PROFILE_PATH, 'Collections/crimes');

function scrapeAlert(ID) {
    if(fs.existsSync(path.join(project, 'maricopa-alert-' + ID + '.json'))) {
        return;
    }
    return client
        .url('https://www.maricopacountyattorney.org/CivicAlerts.aspx?AID=' + ID)
        .getAllXPath({
            time: '//*[@class = "single"]//*[@class = "date"]//text()',
            title: '//*[contains(@class, "single")]//h3//text()',
            content: '//*[@class = "single"]//*[@class = "content"]//text()'
        })
        .then(r => {
            fs.writeFileSync(path.join(project, 'maricopa-alert-' + ID + '.json'), JSON.stringify(r, null, 4));
            return r;
        })
        .catch(e => console.log(e))
}
module.exports = scrapeAlert;

if(typeof $$ !== 'undefined') {
    $$.async();
    var IDs = Array.from(Array(500).keys()).map(i => );
    multiCrawl(IDs, 'crime reports')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


multi crawl?


In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var CONNECTIONS = 10;

function multiCrawl(arr, search) {
    var contexts = [];
    return importer.runAllPromises(Array.from(Array(CONNECTIONS).keys()).map((s, i) => resolve => {
        const client = runSeleniumCell(false, false);
        const ctx = {client};
        const funcs = importer.import([
            'decrypt password',
            'all elements xpath',
            'get all elements until',
            'resize selenium window',
            'click spa link'
        ], ctx);
        contexts[i] = Object.assign(ctx, funcs);
        return client
            .catch(e => console.log(e))
            .then(r => resolve())
    }))
        .then(() => importer.import([
                'resize selenium window',
                'get all session urls'
            ], contexts[0]).getAllSessionUrls())
        .then(() => {
            const promises = [];
            for(var j = 0; j < arr.length / CONNECTIONS; j++) {
                promises.push((j => resolve => Promise.all(contexts.map((ctx, i) => {
                    const callback = importer.import(search, ctx);
                    if(arr.length < j * CONNECTIONS + i) {
                        return;
                    }
                    return callback(arr[j * CONNECTIONS + i]);
                })).then(r => resolve(r)))(j));
            }
            return importer.runAllPromises(promises);
        })
}
module.exports = multiCrawl;



search results as json?

In [ ]:
function searchResultsToJson(url) {
    console.log(url);
    return client
        .url(url)
        .pause(2000)
        .getAllXPath({
            query: '//input[contains(@aria-label, "Search")]/@value'
            +            
            '|//input[contains(@aria-label, "search")]/@value'
            +
            // yahoo
            '|//label[contains(., "Search")]/following::*//input[@type="text"]/@value' 
            +
            '|//input[contains(@class, "Search")]/@value'
            +
            // wolfram
            '|//input[contains(@name, "query")]/@value'
            +
            // duckduckgo
            '|//input[contains(@id, "search")]/@value'
            +
            // yandex
            '|//input[contains(@aria-label, "Request")]/@value',
            results: [
                '//h3|//h2|div[contains(@class, "title")]'
                +
                // ask
                '|//*[contains(@class, "item-title")]',
                {
                    name: './/text()',
                    summary: './/following-sibling::div//text()'
                }
            ]
        })
        .then(r => {
            return {
                url: url,
                query: typeof r.query === 'string'
                    ? r.query
                    : r.query[0],
                results: r.results.map(s => ({
                    name: typeof s.name === 'string'
                        ? s.name : s.name.join('\n'),
                    summary: typeof s.summary === 'string'
                        ? s.summary : s.summary.join('\n')
                }))
            };
        })
        .catch(e => console.log(e))
}
module.exports = searchResultsToJson;


search all?


In [5]:
var importer = require('../Core');
var multiCrawl = importer.import('multi crawl');

// http://www.exploratorium.edu/files/ronh/research/index.html
/*

 Exploratorium	
 Search
 Google	
 Search
 alltheweb	
 Search
 Teoma	
 Search
 AltaVista	
 Search
 Wisenut	
 Search
 HotBot	
 Search
 lii.org	
 Search
 Northern Light	
 Search
 Lycos	
 Search
 Scirus	
 
 Meta:
  Ask Jeeves	
 Search
 MetaCrawler	
 Search
 Dogpile	
 Search
 SavvySearch	
 
 
 */

function searchAll(query = 'search engine') {
    var engines = [
        'https://www.google.com/search?q=' + query,
        'https://www.bing.com/search?q=' + query,
        'https://search.yahoo.com/search?p=' + query,
        'https://www.ask.com/web?q=' + query,
        'https://search.aol.com/aol/search?q=' + query,
        'http://www.baidu.com/s?wd=' + query,
        'https://www.wolframalpha.com/input/?i=' + query,
        'https://duckduckgo.com/?q=' + query,
        'https://www.yandex.com/search/?text=' + query,
        'https://archive.org/search.php?query=' + query,
    ];
    
    return multiCrawl(engines, 'search results json');
}
module.exports = searchAll;

if(typeof $$ !== 'undefined') {
    $$.async();
    searchAll()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


Initializing webdriver on localhost
[10:34:58]  COMMAND	GET 	 "/wd/hub/session/ebc8470bcd9603d3a26827245b3f636f/window_handle"
[10:34:58]  COMMAND	GET 	 "/wd/hub/session/c099015f5fa82fc7f786034f534adf15/window_handle"
[10:34:59]  COMMAND	GET 	 "/wd/hub/session/95971e68edbd027d8ba037c945113c24/window_handle"
[10:34:59]  COMMAND	GET 	 "/wd/hub/session/96ecadc6537a1db3ad86c8d2e6334717/window_handle"
[10:34:59]  COMMAND	GET 	 "/wd/hub/session/d7183aeb14144d540840bf71e9bae394/window_handle"
[10:34:59]  COMMAND	GET 	 "/wd/hub/session/594b33ed4b2103d5c891b9c772629a6c/window_handle"
[10:34:59]  COMMAND	GET 	 "/wd/hub/session/2bee8d47afed978611919bf49b846c3f/window_handle"
[10:34:59]  COMMAND	GET 	 "/wd/hub/session/fe3fda99db8af99e33b3b9d398e29d94/window_handle"
[10:34:59]  COMMAND	GET 	 "/wd/hub/session/a6e50e0427a41bfcf2073fffc941f268/window_handle"
[10:34:59]  COMMAND	POST 	 "/wd/hub/session/a6e50e0427a41bfcf2073fffc941f268/window"
[10:34:59]  COMMAND	GET 	 "/wd/hub/session/a6e50e0427a41bfcf

[ { url: 'https://www.google.com/search?q=search engine',
    query: 'search engine',
    results: 
     [ [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object] ] },
  { url: 'https://www.bing.com/search?q=search engine',
    query: 'search engine',
    results: 
     [ [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object] ] },
  { url: 'https://search.yahoo.com/search?p=search engine',
    query: 'search engine',
    results: 
     [ [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Obje